# SELECT {fields} FROM {table} WHERE {condition}

- plus use of LIKE and wildcards in text fields.
- pluse use of DISTINCT keyword to ensures each col shows up once.
- Once connection is established and the query is defined, can convert this query in a DF, using (at least) to methods:
1. mk the cursor query and convert the resultant list in DF
2. mk the query directly from pandas: pd.
- PANDAS: mk the same query but using ONLY pandas (first import the table as a df)
- Plus: https://www.freecodecamp.org/news/connect-python-with-sql/

## First establish the connection conecting w/the DB

In [1]:
### Connect to the DB - Establish the connection
import pyodbc

# Valid values for the connection string
driver = '{ODBC Driver 17 for SQL Server}'
server = '(local)'
dbname = 'AdventureWorks2019'
#dbname = 'BikeStores'
user = 'user1'
passwd = 'pass1'

# Construct the Connection String
connection_string = f'DRIVER={driver};SERVER={server};' +\
                    f'DATABASE={dbname};UID={user};PWD={passwd}'
print('Connection String:\n', connection_string)

# Establish the connection
try:
    connection = pyodbc.connect(connection_string)
    cursor = connection.cursor()
    print('SUCCESS: Connection Established')
except pyodbc.Error as e:
    print('ERROR:', e)

Connection String:
 DRIVER={ODBC Driver 17 for SQL Server};SERVER=(local);DATABASE=AdventureWorks2019;UID=user1;PWD=pass1
SUCCESS: Connection Established


## Using SQL Server and Quering directly to de DB using cursor

In [2]:
# mk functions to convert SQL queries to DF
import pandas as pd

def df_from_query(qry):     # convert cursor.execute(query) to DF
    cursor.execute(qry)
    field_names = [i[0] for i in cursor.description]
    get_data = [list(x) for x in cursor]
    df = pd.DataFrame(data=get_data, columns=field_names)
    return df

def df_from_fetchall(qry):
    cursor.execute(qry)
    results = cursor.fetchall()
    from_db = [list(r) for r in results]
    cols = [i[0] for i in cursor.description]
    df = pd.DataFrame(data=from_db, columns=cols)
    return df

# cols = field_names =? cursor.keys()

In [3]:
### First query, first look to the products table
query1 = ''' SELECT *
            FROM Production.Product; '''

products_table_df = df_from_query(query1)

print(query1)
display(products_table_df.iloc[[0, 5, -5, -1]])
products_table_df.columns   # to see all cols name of the table

 SELECT *
            FROM Production.Product; 


,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,...,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0000,0.0000,...,None,None,None,NaN,NaN,2008-04-30,NaT,None,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
5,317,LL Crankarm,CA-5965,False,False,Black,500,375,0.0000,0.0000,...,None,L,None,NaN,NaN,2008-04-30,NaT,None,3C9D10B7-A6B2-4774-9963-C19DCEE72FEA,2014-02-08 10:01:36.827
499,995,ML Bottom Bracket,BB-8107,True,True,None,500,375,44.9506,101.2400,...,None,M,None,5.0,96.0,2013-05-30,NaT,None,71AB847F-D091-42D6-B735-7B0C2D82FC84,2014-02-08 10:01:36.827
503,999,"Road-750 Black, 52",BK-R19B-52,True,True,Black,100,75,343.6496,539.9900,...,R,L,U,2.0,31.0,2013-05-30,NaT,None,AE638923-2B67-4679-B90E-ABBAB17DCA31,2014-02-08 10:01:36.827


Index(['ProductID', 'Name', 'ProductNumber', 'MakeFlag', 'FinishedGoodsFlag',
       'Color', 'SafetyStockLevel', 'ReorderPoint', 'StandardCost',
       'ListPrice', 'Size', 'SizeUnitMeasureCode', 'WeightUnitMeasureCode',
       'Weight', 'DaysToManufacture', 'ProductLine', 'Class', 'Style',
       'ProductSubcategoryID', 'ProductModelID', 'SellStartDate',
       'SellEndDate', 'DiscontinuedDate', 'rowguid', 'ModifiedDate'],
      dtype='object')

In [4]:
### Find a type of product entered by user w/ListPrice > 0

# Input an string to find in product name
prod_to_find = input('Enter a product to find:')
# To find the string in every place of the text field
str_to_find = '%' + prod_to_find + '%'
#print(str_to_find)
query2 = ''' SELECT ProductID, Name, ListPrice, SellEndDate, DiscontinuedDate
            FROM Production.Product
            WHERE ListPrice > -1 AND Name LIKE ''' + "'" + str_to_find + "'"
# query is only an string, in this case a python sting, then is better:
query2 =f''' SELECT ProductID, Name, ListPrice, SellEndDate, DiscontinuedDate
            FROM Production.Product
            WHERE ListPrice > -1 AND Name LIKE '%{prod_to_find}%' '''

product_type_df = df_from_query(query2)

print(query2)
product_type_df

 SELECT ProductID, Name, ListPrice, SellEndDate, DiscontinuedDate
            FROM Production.Product
            WHERE ListPrice > -1 AND Name LIKE '%bike%'


,ProductID,Name,ListPrice,SellEndDate,DiscontinuedDate
0,879,All-Purpose Bike Stand,159.0000,NaT,None
1,877,Bike Wash - Dissolver,7.9500,NaT,None
2,876,Hitch Rack - 4-Bike,120.0000,NaT,None
3,710,"Mountain Bike Socks, L",9.5000,2012-05-29,None
4,709,"Mountain Bike Socks, M",9.5000,2012-05-29,None


In [7]:
# trying df_from_fetchall w/same query2
ptype_df = df_from_fetchall(query2)
ptype_df

,ProductID,Name,ListPrice,SellEndDate,DiscontinuedDate
0,879,All-Purpose Bike Stand,159.0000,NaT,None
1,877,Bike Wash - Dissolver,7.9500,NaT,None
2,876,Hitch Rack - 4-Bike,120.0000,NaT,None
3,710,"Mountain Bike Socks, L",9.5000,2012-05-29,None
4,709,"Mountain Bike Socks, M",9.5000,2012-05-29,None


In [5]:
### DISTINCT https://www.mssqltips.com/sqlservertip/6810/sql-select-distinct-examples/
# query3 = ''' SELECT DISTINCT ListPrice, ProductID, Name, SellEndDate, DiscontinuedDate
query3 = ''' SELECT DISTINCT Color, ListPrice
            FROM Production.Product
            WHERE Name LIKE 'Mountain-500%' '''
diffprice_mountain_df = df_from_query(query3)

print(query3)
diffprice_mountain_df
# Isnt't best example to see DISTINCT but we well practice later

 SELECT DISTINCT Color, ListPrice
            FROM Production.Product
            WHERE Name LIKE 'Mountain-500%' 


,Color,ListPrice
0,Black,539.9900
1,Silver,564.9900


## Using pandas sql method pd.read_sql()

In [6]:
# Same second query
df = pd.read_sql(query2, connection)
df
# UserWarning later in trySQLAlchemy.py

C:\Users\jm\AppData\Local\Temp\ipykernel_14848\1869178363.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query2, connection)


,ProductID,Name,ListPrice,SellEndDate,DiscontinuedDate
0,879,All-Purpose Bike Stand,159.00,NaT,None
1,877,Bike Wash - Dissolver,7.95,NaT,None
2,876,Hitch Rack - 4-Bike,120.00,NaT,None
3,710,"Mountain Bike Socks, L",9.50,2012-05-29,None
4,709,"Mountain Bike Socks, M",9.50,2012-05-29,None


## ONLY PANDAS

In [ ]:
## First import whole tabla to a df - done in query1



- https://datatofish.com/sql-to-pandas-dataframe/
- https://learn.microsoft.com/en-us/sql/machine-learning/data-exploration/python-dataframe-pandas?view=sql-server-ver16
- https://stackoverflow.com/questions/43175382/python-create-a-pandas-data-frame-from-a-list
- https://learn.microsoft.com/en-us/sql/connect/python/pyodbc/step-3-proof-of-concept-connecting-to-sql-using-pyodbc?view=sql-server-ver16
- https://stackoverflow.com/questions/3783238/python-database-connection-close
- https://www.freecodecamp.org/news/connect-python-with-sql/